In [1]:
print(2+3)

5


In [2]:
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import os , json , pandas as pd , traceback , PyPDF2 
from dotenv import load_dotenv


In [3]:
# load keys as environment variables
# it set the .env file and stores all the imp keys
load_dotenv()
key=os.getenv("openai_api_key")

In [4]:
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.5)

In [5]:
with open("D://programs/genAi_project/response.json","r") as f:
    response_json=json.load(f)

In [6]:
Template1="""
Text:{text}
you are an expert MCQ maker.given the above text,it is your job to \
create a quiz on {number} multiple questions for {subject} student in {ton}\
make sure the question are not repeated and check all the question to be conformned\
make sure ot format your response like response_json below and use it as a guide.\
ensure to make {number} MCQ's \
{response_json}
"""

In [7]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","ton","response_json"],
    template=Template1
)

In [8]:
quizChain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_13580\323202012.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quizChain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)


In [9]:
# to check the quiz that we generate is correct or not?
# to evaluate we did.
Template2="""you are an expert english grammarian and writer.given a multiple choice question for {subject}\
    you need to evaluate the complexity of the grammer and give the complete analysis of quiz. only use at max 50 words for coplexity analysis. \
    if the quiz is not at per with the cognitive and analytical abilities of the student,\
    update the quiz question which need to be changed and change the tone such that it perfectly fits the student abilities.\
    Quiz_MCQ's:{quiz}
    check from an expert english writer of the above quiz:
    """

In [10]:
quiz_evalation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=Template2
)

In [11]:
reviewChain=LLMChain(llm=llm,prompt=quiz_evalation_prompt,output_key="review",verbose=True)

In [12]:
#here we join both chain as sequential chain
generate_evaluate_chain=SequentialChain(chains=[quizChain,reviewChain],input_variables=["text","number","subject","ton","response_json"],output_variables=["quiz","review"],verbose=True)

In [13]:
# generate quix based on any data testing model 
with open("D://programs//genAi_project//data.txt","r",encoding="utf-8")as f:
    content=f.read()

In [14]:
content
NUMBER=5
SUBJECT="Ai"
TON="SIMPLE"
RESPONSE=response_json

In [29]:
#based on content we have to generate mcq:
# important method:
with get_openai_callback() as cb:
    result = generate_evaluate_chain({
        "text": content,
        "number": NUMBER,
        "subject": SUBJECT,
        "ton": TON,
        "response_json": json.dumps(RESPONSE)
    })




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:India take a lap around the ground. Massive cheers for Siraj who soaks it all in after an unreal finish. He's not let go of the match ball. A stump's there as a souvenir. He's closed out one of India's greatest Test wins. The series, which has been contested so closely, ends 2-2. Countless what-ifs but reality has surged past them with waves of drama. The final day lasted under ten overs but packed the action that entire series wish they could. It encapsulated this one with Siraj taking over as the leader. He chucked the wobble seam for swing, hit 90mph, beat the bat as regularly as he did yesterday and topped it with wickets. Overton started the day with two boundaries but were then hit by Siraj. Chris Woakes came out to bat in a sling, didn't face a ball but literally ran through the pain and did everything right to keep Atkinson on strike. He did that, but in the end, it was S

In [30]:
result.get("quiz")

'{\n    "1": {\n        "no": "1",\n        "mcq": "Who closed out one of India\'s greatest Test wins in the series?",\n        "options": {\n            "a": "Chris Woakes",\n            "b": "Overton",\n            "c": "Siraj",\n            "d": "Atkinson"\n        },\n        "correct": "c"\n    },\n    "2": {\n        "no": "2",\n        "mcq": "How did Siraj finish the match?",\n        "options": {\n            "a": "With a boundary",\n            "b": "With a yorker for the ages",\n            "c": "By hitting a six",\n            "d": "By running out a batsman"\n        },\n        "correct": "b"\n    },\n    "3": {\n        "no": "3",\n        "mcq": "Who started the final day with two boundaries?",\n        "options": {\n            "a": "Chris Woakes",\n            "b": "Overton",\n            "c": "Siraj",\n            "d": "Atkinson"\n        },\n        "correct": "b"\n    },\n    "4": {\n        "no": "4",\n        "mcq": "Who came out to bat in a sling on the final day

In [16]:
print(f"total token:{cb.total_tokens}")
print(f"prompt token:{cb.prompt_tokens}")
print(f"completion token:{cb.completion_tokens}")
print(f"total cost:{cb.total_cost}")

total token:1445
prompt token:1018
completion token:427
total cost:0.0011495


In [27]:
# here firstly get the result.quiz  in str format and then load into json format 
quiz=json.loads(result.get("quiz"))

In [18]:
# in proper format:
quizdata=[]
for key,value in quiz.items():
    mcq=value["mcq"]
    options=" | ".join(
        [ 
            f"{option}:{option_value}" for option,option_value in value["options"].items()
        ]      
    )
    correct=value["correct"]
    quizdata.append({"MCQ":mcq,"Choices":options,"Correct":correct})

In [19]:
quizdata

[{'MCQ': "Who closed out one of India's greatest Test wins in the series?",
  'Choices': 'a:Chris Woakes | b:Siraj | c:Overton | d:Atkinson',
  'Correct': 'b'},
 {'MCQ': 'How did Siraj finish the match?',
  'Choices': 'a:With a boundary | b:With a yorker | c:With a six | d:With a no-ball',
  'Correct': 'b'},
 {'MCQ': 'Who started the final day with two boundaries?',
  'Choices': 'a:Chris Woakes | b:Siraj | c:Overton | d:Atkinson',
  'Correct': 'c'},
 {'MCQ': 'Who came out to bat in a sling?',
  'Choices': 'a:Chris Woakes | b:Siraj | c:Overton | d:Atkinson',
  'Correct': 'a'},
 {'MCQ': 'Who kept Atkinson on strike towards the end?',
  'Choices': 'a:Chris Woakes | b:Siraj | c:Overton | d:Atkinson',
  'Correct': 'a'}]

In [20]:
df=pd.DataFrame(quizdata)
# df.to_csv("mcqgenerativeAI.csv",index=False)
df

,MCQ,Choices,Correct
0,Who closed out one of India's greatest Test wi...,a:Chris Woakes | b:Siraj | c:Overton | d:Atkinson,b
1,How did Siraj finish the match?,a:With a boundary | b:With a yorker | c:With a...,b
2,Who started the final day with two boundaries?,a:Chris Woakes | b:Siraj | c:Overton | d:Atkinson,c
3,Who came out to bat in a sling?,a:Chris Woakes | b:Siraj | c:Overton | d:Atkinson,a
4,Who kept Atkinson on strike towards the end?,a:Chris Woakes | b:Siraj | c:Overton | d:Atkinson,a


In [21]:
df.head(3)

,MCQ,Choices,Correct
0,Who closed out one of India's greatest Test wi...,a:Chris Woakes | b:Siraj | c:Overton | d:Atkinson,b
1,How did Siraj finish the match?,a:With a boundary | b:With a yorker | c:With a...,b
2,Who started the final day with two boundaries?,a:Chris Woakes | b:Siraj | c:Overton | d:Atkinson,c
